In [4]:
    import tensorflow as tf
import numpy as np
import emnist
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
#from IPython.display import display, Image
from PIL import Image, ImageFilter

import numpy


#subset = [18, 20, 27, 44, 46, 53]               # r,i,k,R,I,
subset = [0,1,2,3,4,5,6,7,8,9]          # 0-9, x, X
#subset = [2,3,6,7,9]

mnist = emnist.read_data_sets('data/emnist', subset = subset)
print mnist.train.images.shape



Extracting data/emnist/emnist-byclass-train-images-idx3-ubyte.gz
Extracting data/emnist/emnist-byclass-train-labels-idx1-ubyte.gz
Extracting data/emnist/emnist-byclass-test-images-idx3-ubyte.gz
Extracting data/emnist/emnist-byclass-test-labels-idx1-ubyte.gz
Only retain label: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Only retain label: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Only retain label: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
(342584, 784)


In [5]:
def weight_variable(shape, name):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial, name=name)

def bias_variable(shape, name):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial, name=name)

def conv2d(x, W, name=None):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME', name=name)

def max_pool_2x2(x, name):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME', name=name)

image_size = 28
num_label = len(subset)
num_steps = 20000

graph = tf.Graph()
print("mnist.train.next_batch(50):" ,mnist.train.next_batch(50)[0].shape)
with graph.as_default():  
    x = tf.placeholder(tf.float32, shape=[None, 784], name='x')
    y_ = tf.placeholder(tf.float32, shape=[None, num_label], name='y_')
    global_step = tf.Variable(0, name='global_step', trainable=False)
       
    W_conv1 = weight_variable([5, 5, 1, 32], 'W_conv1')
    b_conv1 = bias_variable([32], 'b_conv1')
    x_image = tf.reshape(x, [-1, 28, 28, 1], name='x_image')
                        

    h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1, name='h_conv1')
    h_pool1 = max_pool_2x2(h_conv1, 'h_pool1')
    W_conv2 = weight_variable([5, 5, 32, 64], 'W_conv2')
    b_conv2 = bias_variable([64], 'b_conv2')
    h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
    h_pool2 = max_pool_2x2(h_conv2, 'h_pool2')
    W_fc1 = weight_variable([7 * 7 * 64, 1024], 'W_fc1')
    b_fc1 = bias_variable([1024], 'b_fc1')
    h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)
    W_fc2 = weight_variable([1024, num_label], 'W_fc2')
    b_fc2 = bias_variable([num_label], 'b_fc2')

    y_conv = tf.identity(tf.matmul(h_fc1_drop, W_fc2) + b_fc2,name='y_conv')
    
    
    cross_entropy = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv), name='cross_entropy')
    train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy, name='train_step')
    correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name='accuracy')
    tf.add_to_collection('y_conv', y_conv)
    tf.add_to_collection('accuracy', accuracy)
    tf.add_to_collection('cross_entropy', cross_entropy)
    tf.add_to_collection("train_step", train_step)

print('mnist.test_images.shape', mnist.test.images.shape)
print('mnist.test_labels.shape', mnist.test.labels.shape)

trained_filename = '/Users/nghiaround/rec_table/train/checkpoint/0no'
#print('Label ', mnist.test.labels[0:10])
with tf.Session(graph=graph) as session:
    session.run(tf.initialize_all_variables())
    saver = tf.train.Saver()
    for i in range(num_steps):
        batch = mnist.train.next_batch(50)
        if i % 100 == 0:
            train_accuracy = accuracy.eval(feed_dict={
                x: batch[0], y_: batch[1], keep_prob: 1.0})
            print('step %d, training accuracy %g' % (i, train_accuracy))
            
        train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})
    print('test accuracy %g' % accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))
    saver.save(session, trained_filename)
    
    

('mnist.train.next_batch(50):', (50, 784))
('mnist.test_images.shape', (57918, 784))
('mnist.test_labels.shape', (57918, 10))
step 0, training accuracy 0.06
step 100, training accuracy 0.78
step 200, training accuracy 0.86
step 300, training accuracy 0.96
step 400, training accuracy 0.88
step 500, training accuracy 0.96
step 600, training accuracy 0.98
step 700, training accuracy 0.96
step 800, training accuracy 1
step 900, training accuracy 0.98
step 1000, training accuracy 1
step 1100, training accuracy 1
step 1200, training accuracy 0.94
step 1300, training accuracy 1
step 1400, training accuracy 0.98
step 1500, training accuracy 0.98
step 1600, training accuracy 0.94
step 1700, training accuracy 0.98
step 1800, training accuracy 0.98
step 1900, training accuracy 1
step 2000, training accuracy 0.98
step 2100, training accuracy 0.98
step 2200, training accuracy 0.98
step 2300, training accuracy 0.98
step 2400, training accuracy 0.98
step 2500, training accuracy 0.98
step 2600, traini

In [6]:
# Retrain from last checkpoint 
trained_filename = '/Users/nghiaround/rec_table/train/checkpoint/0_9'
new_trained_filename = '/Users/nghiaround/rec_table/train/checkpoint/0_9_local'
num_steps = 10

# We can now access the default graph where all our metadata has been loaded
graph = tf.Graph()
session = tf.Session(graph=graph)
print("Starting retrain...")
with graph.as_default():
    new_saver = tf.train.import_meta_graph(trained_filename + '.meta') 
    new_saver.restore(session, trained_filename)
    x = graph.get_tensor_by_name("x:0") 
    y_ = graph.get_tensor_by_name("y_:0")
    keep_prob = graph.get_tensor_by_name("keep_prob:0")
    train_step = tf.get_collection("train_step")[0]
    accuracy = tf.get_collection('accuracy')[0]
    for i in range(num_steps):
        batch = mnist.train.next_batch(50) 
        if i % 10 == 0:
            train_accuracy = session.run(accuracy, feed_dict={x: batch[0], y_: batch[1], keep_prob: 1.0})
            print('step %d, training accuracy %g' % (i, train_accuracy))
            
        session.run(train_step, feed_dict={'x:0': batch[0], 'y_:0': batch[1], keep_prob: 0.5})
    print('test accuracy %g' % session.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))    
    


Starting retrain...
step 0, training accuracy 1
test accuracy 0.994061


In [ ]:
def imageprepare(filename):
    """
    This function returns the pixel values.
    The imput is a png file location.
    """
    im = Image.open(filename).convert('L')
    width = float(im.size[0])
    height = float(im.size[1])
    newImage = Image.new('L', (28, 28), (0)) #creates white canvas of 28x28 pixels
    
    if width > height: #check which dimension is bigger
        #Width is bigger. Width becomes 20 pixels.
        nheight = int(round((20.0/width*height),0)) #resize height according to ratio width
        if (nheight == 0): #rare case but minimum is 1 pixel
            nheight = 1
        # resize and sharpen
        img = im.resize((20,nheight), Image.ANTIALIAS).filter(ImageFilter.SHARPEN)
        wtop = int(round(((28 - nheight)/2),0)) #caculate horizontal pozition
        newImage.paste(img, (4, wtop)) #paste resized image on white canvas
    else:
        #Height is bigger. Heigth becomes 20 pixels. 
        nwidth = int(round((20.0/height*width),0)) #resize width according to ratio height
        if (nwidth == 0): #rare case but minimum is 1 pixel
            nwidth = 1
         # resize and sharpen
        img = im.resize((nwidth,20), Image.ANTIALIAS).filter(ImageFilter.SHARPEN)
        wleft = int(round(((28 - nwidth)/2),0)) #caculate vertical pozition
        newImage.paste(img, (wleft, 4)) #paste resized image on white canvas
    
    #newImage.save("sample.png")

    tv = list(newImage.getdata()) #get pixel values
    
    #normalize pixels to 0 and 1. 0 is pure white, 1 is pure black.
    tva = [ (x)*1.0/255.0 for x in tv] 
    return tva
    #print(tva)
    
%matplotlib inline 
image_size = 28 

graph = tf.Graph() 
with graph.as_default():
    filename = "/Users/nghiaround/Desktop/out/file229_sub_0.png"
    img = imageprepare(filename)
tt = 900
one_image = mnist.train.images[tt] 
show_image = one_image.reshape([image_size, image_size])

predict_image = np.array(img)
predict_image = predict_image.reshape([image_size, image_size])
plt.imshow(predict_image, aspect="auto") 

#show_image = numpy.flip(one_image.reshape([image_size, image_size]), 0)
#show_image = numpy.rot90(show_image, 3)
graph = tf.Graph() 

#img_input = img.reshape([1, image_size * image_size])
#img_input = one_image.reshape([1, image_size * image_size]).astype(np.float32)
letter_map = ['I', 'K', 'R', 'i', 'k', 'r']
#letter_map = '0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz'
#print("Label:", mnist.train.labels[tt])
#print("Label:", letter_map[np.argmax(mnist.train.labels[tt])])
#plt.imshow(show_image, aspect="auto") 
#print(show_image)


with graph.as_default():
    new_saver = tf.train.import_meta_graph(trained_filename + '.meta') 
    x = graph.get_tensor_by_name("x:0") 
    keep_prob = graph.get_tensor_by_name("keep_prob:0")
    y_conv = tf.get_collection('y_conv')[0]
    predict = tf.argmax(y_conv, 1)
with tf.Session(graph=graph) as session:
    new_saver.restore(session, trained_filename)
    v_ = session.run(predict, feed_dict = {"x:0": [img], "keep_prob:0": 1.0}) 
    #print(v_)
print("Result:", letter_map[v_[0]])
    
    


